In [139]:
import requests

In [140]:
APIKEY = 'RFb2_XmabVtkYj4SFsIdl8oZFc0ApE5F'
symbol = 'AMZN'
date_start_train = '2023-01-01'
date_end_train = '2023-07-01'
date_start_val = '2023-07-02'
date_end_val = '2024-01-01'
date_start_test = '2024-01-02'
date_end_test = '2024-06-01'

In [141]:
url_train = f'https://api.polygon.io/v2/aggs/ticker/{symbol}/range/1/day/{date_start_train}/{date_end_train}?adjusted=true&sort=asc&apiKey={APIKEY}'
url_val = f'https://api.polygon.io/v2/aggs/ticker/{symbol}/range/1/day/{date_start_val}/{date_end_val}?adjusted=true&sort=asc&apiKey={APIKEY}'

response_train = requests.get(url_train)
response_val = requests.get(url_val)


In [142]:
# Run this 1 minuet after the cell above because I am using the free subscription.

url_test = f'https://api.polygon.io/v2/aggs/ticker/{symbol}/range/1/day/{date_start_test}/{date_end_test}?adjusted=true&sort=asc&apiKey={APIKEY}'
response_test = requests.get(url_test)



In [143]:
if response_train.status_code == 200:
    data = response_train.json()
    stock_data_train = data['results']
    data = response_val.json()
    stock_data_val = data['results']
    data = response_test.json()
    stock_data_test = data['results']
else:
    print(f'Error: {response.status_code}')

In [147]:
print(stock_data_test)
# print(len(stock_data_val))
# print(len(stock_data_test))

[{'v': 47314924.0, 'vw': 149.8529, 'o': 151.54, 'c': 149.93, 'h': 152.38, 'l': 148.39, 't': 1704171600000, 'n': 476432}, {'v': 49410095.0, 'vw': 149.4055, 'o': 149.2, 'c': 148.47, 'h': 151.05, 'l': 148.33, 't': 1704258000000, 'n': 416954}, {'v': 56039807.0, 'vw': 145.416, 'o': 145.59, 'c': 144.57, 'h': 147.38, 'l': 144.05, 't': 1704344400000, 'n': 541342}, {'v': 45003147.0, 'vw': 145.4485, 'o': 144.69, 'c': 145.24, 'h': 146.59, 'l': 144.53, 't': 1704430800000, 'n': 380385}, {'v': 46757053.0, 'vw': 148.3238, 'o': 146.74, 'c': 149.1, 'h': 149.4, 'l': 146.15, 't': 1704690000000, 'n': 410733}, {'v': 43812567.0, 'vw': 150.6311, 'o': 148.33, 'c': 151.37, 'h': 151.71, 'l': 148.21, 't': 1704776400000, 'n': 403026}, {'v': 44421830.0, 'vw': 153.4523, 'o': 152.06, 'c': 153.73, 'h': 154.42, 'l': 151.881, 't': 1704862800000, 'n': 430517}, {'v': 49072691.0, 'vw': 155.0181, 'o': 155.04, 'c': 155.18, 'h': 157.17, 'l': 153.12, 't': 1704949200000, 'n': 468131}, {'v': 40455355.0, 'vw': 154.8237, 'o': 155

In [148]:
# This removes the unnecessary features in the data returned from the api. 
def clean_stock_data(data):
    for day in data:
        del day['t'] 
        del day['v']
        del day['n'] 
        del day['vw']
    return data

# Assuming you have your splits already defined
train_data = clean_stock_data(stock_data_train)
val_data = clean_stock_data(stock_data_val)
test_data = clean_stock_data(stock_data_test)







In [149]:
print(len(train_data))
print(len(val_data))
print(len(test_data))

124
126
105


In [150]:
import pandas as pd
from sklearn.preprocessing import MinMaxScale

print(len(train_data))
print(len(val_data))
print(len(test_data))


def normalize_data(data):
    # To normalize the data we have to convert the dictionaries into a dataframe, then back into a list of dictionaries
    
    # Step 1: Convert the list of dictionaries into a dataframe
    df = pd.DataFrame(data)
    
    # Step 2: Convert the normalized array back to a DataFrame with the original column names
    scaler = MinMaxScaler(feature_range=(0, 1))
    normalized_array = scaler.fit_transform(df) # This converts it into a array
    
    # Convert the normalized array back to a DataFrame with the original column names
    normalized_df = pd.DataFrame(normalized_array, columns=df.columns)
    
    # Step 3: Convert the DataFrame back to a list of dictionaries
    normalized_stock_data = normalized_df.to_dict(orient='records') # This is the new variable containing all of the data.

    return normalized_stock_data

# Apply everything to the data
train_data = normalize_data(train_data)
val_data = normalize_data(val_data)
test_data = normalize_data(test_data)



# print(train_data)

124
126
105


In [153]:
import numpy as np
# This is what the model will train on. The remaining of the data is like the y values.
def create_sequences_and_targets(data, window_size):
    sequences = []
    targets = []
    for i in range(len(data) - window_size):
        # Extract 'o', 'h', 'l' for each day excluding 'c'
        sequence = [[day['o'], day['h'], day['l']] for day in data[i:i+window_size]]
        sequences.append(sequence) # X data
        targets.append(data[i+window_size]['c']) # y data
    return np.array(sequences), np.array(targets)

# Set your window size (e.g., 60 days)
window_size = 40

# Create sequences
train_sequences, train_targets = create_sequences_and_targets(train_data, window_size)
val_sequences, val_targets = create_sequences_and_targets(val_data, window_size)
test_sequences, test_targets = create_sequences_and_targets(test_data, window_size)

print(train_sequences.shape)  
print(train_targets.shape) 

print(val_sequences.shape)
print(val_targets.shape)

print(test_sequences.shape)
print(test_targets.shape)

(84, 40, 3)
(84,)
(86, 40, 3)
(86,)
(65, 40, 3)
(65,)


In [169]:
print(test_targets)

[0.71644781 0.7489428  0.73469842 0.65768974 0.64411306 0.71778322
 0.68506566 0.60961496 0.68595593 0.71188515 0.76073893 0.66436679
 0.6657022  0.69730692 0.74738482 0.74738482 0.76340975 0.7821055
 0.75072335 0.78477632 0.79701758 0.81014912 0.8039172  0.84219898
 0.78855998 0.90140218 0.904073   0.91475629 0.9209882  0.98998442
 0.92499444 0.86912976 0.8624527  0.81704874 0.77119964 0.66904073
 0.72690852 0.77832183 0.71266414 0.64767416 0.78010238 0.80992655
 0.67727576 0.76630314 0.89361229 0.92677498 0.98219452 0.98352994
 0.96661473 1.         0.95504118 0.93478745 0.94591587 0.92187848
 0.86935233 0.89316715 0.86734921 0.85866904 0.8582239  0.81192967
 0.80525262 0.8364122  0.83351881 0.77342533 0.70932562]


In [172]:
# This is to convert the dictionaries into a format to where it could be fed into the model. 

#transformed_sequences = []

# Iterate through the 113 sequences
#for sequence in sequences:
    #transformed_sequence = []
    # Iterate each day in the sequence
    #for day in sequence:
        # Extract values from the dictionary and create a list for each day
        #transformed_day = [day['o'], day['c'], day['h'], day['l']]
        #transformed_sequence.append(transformed_day)
    #transformed_sequences.append(transformed_sequence)

# Convert the list of lists into a numpy array
#transformed_sequences = np.array(transformed_sequences)

#print(transformed_sequences.shape) #The shape is (113, 40, 4) because there are 113 sequences, with each sequence having 40 days worth of data, and each day had 4 features (open, close, high, low). Each of these are nested into eachother. 


print(val_sequences.shape)
print(val_targets.shape)

(86, 40, 3)
(86,)


In [176]:
from keras.models import Sequential
from keras.layers import LSTM, Dense, Dropout
from keras.callbacks import EarlyStopping
from keras.optimizers import Adam
from keras.callbacks import ModelCheckpoint


model = Sequential()
model.add(LSTM(units=100, return_sequences=True, input_shape=(window_size, 3)))
model.add(Dropout(0.2))
model.add(LSTM(units=100, return_sequences=True))
model.add(Dropout(0.2))
model.add(LSTM(units=100))
model.add(Dropout(0.2))
model.add(Dense(units=1))


optimizer = Adam(learning_rate=0.001)
model.compile(optimizer=optimizer, loss='mean_squared_error')

early_stopping = EarlyStopping(monitor='val_loss', patience=10, mode='min', restore_best_weights=True)
checkpoint = ModelCheckpoint('best_model.h5', monitor='val_loss', save_best_only=True, mode='min')


# Train the model
history = model.fit(train_sequences, train_targets, epochs=500, batch_size=16,
                    validation_data=(val_sequences, val_targets), callbacks=[early_stopping, checkpoint])

Epoch 1/500


2024-08-14 14:19:25.154038: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
2024-08-14 14:19:25.365813: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
2024-08-14 14:19:25.740342: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
2024-08-14 14:19:25.790836: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
2024-08-14 14:19:26.152132: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
2024-08-14 14:19:26.394383: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
2024-08-14 14:19:26.480282: I tensorflow/core/grappler/optimizers/cust

6/6 [==============================] - ETA: 0s - loss: 0.1221

2024-08-14 14:19:27.722018: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
2024-08-14 14:19:27.786376: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
2024-08-14 14:19:27.827450: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
2024-08-14 14:19:27.883394: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


6/6 [==============================] - 4s 332ms/step - loss: 0.1221 - val_loss: 0.0362
Epoch 2/500
6/6 [==============================] - 0s 24ms/step - loss: 0.0326 - val_loss: 0.0368


/Users/hadibishr/miniforge3/envs/stockprediction/lib/python3.8/site-packages/keras/src/engine/training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


Epoch 3/500
6/6 [==============================] - 0s 34ms/step - loss: 0.0134 - val_loss: 0.0368
Epoch 4/500
6/6 [==============================] - 0s 30ms/step - loss: 0.0098 - val_loss: 0.0349
Epoch 5/500
6/6 [==============================] - 0s 31ms/step - loss: 0.0091 - val_loss: 0.0366
Epoch 6/500
6/6 [==============================] - 0s 35ms/step - loss: 0.0067 - val_loss: 0.0315
Epoch 7/500
6/6 [==============================] - 0s 30ms/step - loss: 0.0074 - val_loss: 0.0310
Epoch 8/500
6/6 [==============================] - 0s 32ms/step - loss: 0.0063 - val_loss: 0.0306
Epoch 9/500
6/6 [==============================] - 0s 31ms/step - loss: 0.0052 - val_loss: 0.0302
Epoch 10/500
6/6 [==============================] - 0s 25ms/step - loss: 0.0049 - val_loss: 0.0307
Epoch 11/500
6/6 [==============================] - 0s 33ms/step - loss: 0.0046 - val_loss: 0.0299
Epoch 12/500
6/6 [==============================] - 0s 26ms/step - loss: 0.0045 - val_loss: 0.0300
Epoch 13/500
6/6 

In [177]:
test_loss = model.evaluate(test_sequences, test_targets)

print(f"Test Loss: {test_loss}")

3/3 [==============================] - 0s 136ms/step - loss: 0.0098
Test Loss: 0.009765306487679482


In [178]:
date_start_pred = '2024-06-06'
date_end_pred = '2024-08-03'

url_pred = f'https://api.polygon.io/v2/aggs/ticker/{symbol}/range/1/day/{date_start_pred}/{date_end_pred}?adjusted=true&sort=asc&apiKey={APIKEY}'

response_pred = requests.get(url_pred)

In [179]:
data = response_pred.json()
pred_data = data['results']
pred_data = clean_stock_data(pred_data)

normalized_pred_data = normalize_data(pred_data)

print(len(normalized_pred_data))

40


In [180]:
# Convert to the same format as your training data (list of lists)
pred_data = [[day['o'], day['h'], day['l']] for day in normalized_pred_data]


In [181]:
pred_data = np.array(pred_data).reshape(1, 40, 3)

In [182]:
print(pred_data.shape)

(1, 40, 3)


In [183]:
prediction = model.predict(pred_data)

2024-08-14 14:20:00.976469: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
2024-08-14 14:20:01.048254: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
2024-08-14 14:20:01.082240: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
2024-08-14 14:20:01.115119: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


1/1 [==============================] - 0s 492ms/step


In [184]:
unnormalized_prediction = scaler.inverse_transform([[0, prediction[0][0], 0, 0]])[0][1]

In [185]:
print(f"Predicted Closing Price: {unnormalized_prediction}")


Predicted Closing Price: 202.5430045235157
